- 该笔记主要是对数据做初步分析并进行处理，为后续模型训练准备好数据
- 主要利用了两个数据
    - ml-latest: movielens 数据集 (http://files.grouplens.org/datasets/movielens/),评分数据在20M左右
    - movie_detail.csv，这部分数据是信息比较全的电影数据，为了项目上线，ml-latest只取该数据集里面的电影数据分析，数据来源：https://github.com/LuckyZXL2016/Movie_Recommend

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from IPython.core.display import Image, display
%matplotlib inline

In [4]:
def show_image(url):
    display(Image(url, width=200, unconfined=True))

#### 将数据库导出的表中电影名重名、空值的记录过滤掉

In [11]:
df_movie = pd.read_csv('../../data/movie_detail.csv', engine='python', index_col=0)

In [12]:
df_movie.shape

(50605, 13)

In [13]:
df_movie['moviename'] = df_movie['moviename'].map(lambda x : x.strip() if type(x) is str else np.nan)

In [14]:
df_movie = df_movie.drop_duplicates(subset=['moviename'])

In [15]:
df_movie = df_movie.dropna(subset=['moviename'])

In [16]:
df_movie.shape

(46922, 13)

#### 将movielens的数据集中title重名为空的记录过滤掉

In [18]:
df_movie_latest = pd.read_csv('../../data/ml-latest/movies.csv')

In [19]:
def get_movie_name_and_year(title):
    title = title.strip()
    if '(' not in title:
        return title, ""
    regex = r'^(.*)\((\d+)\)$'
    result = re.match(regex, title)
    if result is None:
        #print(title)
        return title, ""
    else:
        return result.group(1).strip(), result.group(2).strip()

In [20]:
df_movie_latest['moviename'] = df_movie_latest['title'].map(lambda x : get_movie_name_and_year(x)[0])
df_movie_latest['showyear'] = df_movie_latest['title'].map(lambda x : get_movie_name_and_year(x)[1])

In [21]:
df_movie_latest.shape

(58098, 5)

In [22]:
df_movie_latest = df_movie_latest.drop_duplicates(subset=['moviename'])

In [23]:
df_moive_latest = df_movie_latest.dropna(subset=['moviename'])

In [24]:
df_movie_latest.shape

(54636, 5)

In [25]:
df_movie_clean = pd.merge(df_movie, df_movie_latest, on='moviename')

In [26]:
df_movie_clean['showyear'] = df_movie_clean['showyear_x']

In [27]:
df_movie_clean = df_movie_clean[['moviename', 'director',
       'leadactors', 'screenwriter', 'picture', 'averating', 'numrating',
       'description', 'typelist', 'backpost', 'movieId', 'title', 'genres',
       'showyear']]

In [28]:
df_movie_clean = df_movie_clean.reset_index(drop=True)

In [31]:
df_movie_clean.to_csv('../../data/ml-clean/movies.csv')

In [33]:
df_ratings = pd.read_csv('../../data/ml-latest/ratings.csv')

In [34]:
df_ratings_clean = pd.merge(df_movie_clean[['movieId']], df_ratings, on='movieId')

In [35]:
df_ratings_clean.shape

(16205120, 4)

In [36]:
df_ratings.shape

(27753444, 4)

In [37]:
df_ratings_clean = df_ratings_clean.reset_index(drop=True)

In [38]:
df_ratings_clean.to_csv('../../data/ml-clean/ratings.csv')

In [39]:
df_ratings_clean.shape

(16205120, 4)

## 分析数据并作处理

In [41]:
df_movies = pd.read_csv('../../data/ml-clean/movies.csv', index_col=0)

In [42]:
print('电影数量：{}'.format(df_movies.shape[0]))

电影数量：34624


In [43]:
df_ratings = pd.read_csv('../../data/ml-clean/ratings.csv', index_col=0)

In [44]:
df_ratings.columns

Index(['movieId', 'userId', 'rating', 'timestamp'], dtype='object')

In [45]:
print('用户的数量：{}'.format(df_ratings['userId'].nunique()))

用户的数量：278153


In [46]:
print('评论数量：{}'.format(df_ratings.shape[0]))

评论数量：16205120


#### 电影被用户评论的情况

In [47]:
df_ratings['movieId'].value_counts().describe()

count    31468.000000
mean       514.971400
std       2808.592065
min          1.000000
25%          2.000000
50%          6.000000
75%         44.000000
max      97040.000000
Name: movieId, dtype: float64

- 一半的电影评论的用户数不足6个，$\frac{3}{4}$的用户被不到50个用户打分
- 这种很少有用户评论的电影在UserCF中由于过于稀疏，商品之间难以产生关系，需要将评论数小于等于6个的电影去掉

In [48]:
df_valid_movies = df_ratings['movieId'].value_counts().reset_index()

In [49]:
df_valid_movies.columns = ['movieId', 'Count']

In [50]:
df_valid_movies = df_valid_movies[df_valid_movies['Count'] > 6]

#### 用户评论电影的情况

In [51]:
df_ratings['userId'].value_counts().describe()

count    278153.000000
mean         58.259735
std         127.313355
min           1.000000
25%           9.000000
50%          18.000000
75%          55.000000
max       12816.000000
Name: userId, dtype: float64

In [52]:
df_ratings['userId'].value_counts().clip(5000, 20000).describe()

count    278153.000000
mean       5000.029933
std          14.851323
min        5000.000000
25%        5000.000000
50%        5000.000000
75%        5000.000000
max       12816.000000
Name: userId, dtype: float64

- 差不多1/4的用户评论数在10个以下，过少评论的用户提供的信息也比较少，需要去掉。

In [53]:
df_valid_users = df_ratings['userId'].value_counts().reset_index()

df_valid_users.columns = ['userId', 'Count']

df_valid_users = df_valid_users[df_valid_users['Count'] > 10]

#### 多次评论的情况

In [54]:
pd.to_datetime(df_ratings['timestamp'].min(), unit='s')

Timestamp('1995-01-09 11:46:49')

In [55]:
pd.to_datetime(df_ratings['timestamp'].max(), unit='s')

Timestamp('2018-09-26 06:58:50')

In [56]:
df_ratings['movie_user'] = df_ratings['movieId'].map(str) + '_' + df_ratings['userId'].map(str)

In [57]:
df_ratings['movie_user'].value_counts().describe()

count    16205120.0
mean            1.0
std             0.0
min             1.0
25%             1.0
50%             1.0
75%             1.0
max             1.0
Name: movie_user, dtype: float64

In [58]:
df_valid_ratings = df_ratings['movie_user'].value_counts().reset_index()

In [59]:
df_valid_ratings.columns = ['userId', 'Count']

In [60]:
df_valid_ratings = df_valid_ratings[df_valid_ratings['Count'] <= 1]

In [61]:
df_valid_ratings['userId'] = df_valid_ratings['userId'].map(lambda x : int(x.split('_')[0]))

In [62]:
df_valid_ratings = df_valid_ratings.drop_duplicates(subset=['userId'])

In [63]:
df_valid_ratings.shape

(31468, 2)

- 有将近1/10的评论是用户对电影的重复评论
- 将有重复评论的用户筛掉

## 筛选数据

In [64]:
df_ratings_clean = pd.merge(df_ratings, df_valid_movies[['movieId']], on='movieId')

In [65]:
df_ratings_clean = pd.merge(df_ratings_clean, df_valid_users[['userId']], on='userId')

In [66]:
print("筛选前的评论数：{}".format(df_ratings.shape[0]))

筛选前的评论数：16205120


In [67]:
print("筛选后的评论数：{}".format(df_ratings_clean.shape[0]))

筛选后的评论数：15654592


In [68]:
print("筛选前用户数：{}, 筛选后用户数：{}".format(df_ratings['userId'].nunique(), df_ratings_clean['userId'].nunique()))

筛选前用户数：278153, 筛选后用户数：189911


In [69]:
print("筛选前电影数：{}, 筛选后电影数：{}".format(df_ratings['movieId'].nunique(), df_ratings_clean['movieId'].nunique()))

筛选前电影数：31468, 筛选后电影数：15171


In [70]:
df_movies_clean = pd.merge(df_movies, df_valid_movies[['movieId']], on='movieId')

In [71]:
df_valid_movies.shape

(15171, 2)

In [72]:
df_movies.shape

(34624, 14)

In [73]:
df_movies['movieId'].nunique()

34624

## 将筛选后的数据进行保存，供后续模型训练使用

In [4]:
df_movies_clean.to_csv('../../data/ml-clean/movies_v2.csv')

NameError: name 'df_movies_clean' is not defined

In [76]:
df_ratings_clean.to_csv('../../data/ml-clean/ratings_v2.csv')

## 筛选出需要导入微信小程序后台的数据

In [14]:
df_movies_import = pd.read_csv('../../data/ml-clean/movies_v2.csv', index_col=0)

In [15]:
df_movies_import['name'] = df_movies_import['moviename']
df_movies_import['picUrl'] = df_movies_import['picture'].map(lambda x : x if x is not np.nan and 'jpg' in x else np.nan)

In [16]:
df_movies_import['score'] = df_movies_import['averating']

In [17]:
cols = ['movieId', 'name', 'score', 'picUrl', 'director', 'leadactors', 'genres', 'showyear']

In [18]:
df_movies_import = df_movies_import[cols].dropna(subset=['picUrl']).copy()

In [19]:
df_movies_import['imdbPicUrl'] = df_movies_import['picUrl']

In [20]:
df_movies_import['smallPicUrl'] = df_movies_import['picUrl']

In [21]:
df_movies_import.head()

,movieId,name,score,picUrl,director,leadactors,genres,showyear,imdbPicUrl,smallPicUrl
0,1,Toy Story,3.9,http://image.tmdb.org/t/p/w185/rhIRbceoE9lR4ve...,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal...",Adventure|Animation|Children|Comedy|Fantasy,1995-10-30,http://image.tmdb.org/t/p/w185/rhIRbceoE9lR4ve...,http://image.tmdb.org/t/p/w185/rhIRbceoE9lR4ve...
1,2,Jumanji,3.2,http://image.tmdb.org/t/p/w185/8wBKXZNod4frLZj...,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra...",Adventure|Children|Fantasy,1995-12-15,http://image.tmdb.org/t/p/w185/8wBKXZNod4frLZj...,http://image.tmdb.org/t/p/w185/8wBKXZNod4frLZj...
2,3,Grumpier Old Men,3.2,http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...",Comedy|Romance,1995-12-22,http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...,http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...
3,4,Waiting to Exhale,2.9,http://image.tmdb.org/t/p/w185/16XOMpEaLWkrcPq...,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,...",Comedy|Drama|Romance,1995-12-22,http://image.tmdb.org/t/p/w185/16XOMpEaLWkrcPq...,http://image.tmdb.org/t/p/w185/16XOMpEaLWkrcPq...
4,5,Father of the Bride Part II,3.1,http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl...",Comedy,1995-02-10,http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...,http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...


In [22]:
df_movies_import['imdbPicUrl'].head().values.tolist()

['http://image.tmdb.org/t/p/w185/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
 'http://image.tmdb.org/t/p/w185/8wBKXZNod4frLZjAKSDuAcQ2dEU.jpg',
 'http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg',
 'http://image.tmdb.org/t/p/w185/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg',
 'http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7naBFyssKFxVd.jpg']

In [23]:
df_movies_import.to_csv('../../data/wxcloud/import_movies_data.csv', index=None)